In [1]:
# Environment configuration
ENVIRONMENT = 'local'  # Change to 'kaggle' when running on Kaggle

In [2]:
%pip install torch transformers pandas numpy scikit-learn tqdm biopython matplotlib seaborn pyarrow -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from tqdm.auto import tqdm
from Bio import SeqIO
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from collections import Counter
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook', font_scale=1.2)
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Imports successful | Device: {device}")

✅ Imports successful | Device: cuda


In [4]:
# Set base directory
if ENVIRONMENT == 'kaggle':
    base_dir = Path("/kaggle/input/cafa-6-dataset")
else:
    # Handle both running from notebooks/ and project root
    if Path.cwd().name == 'notebooks':
        base_dir = Path.cwd().parent
    else:
        base_dir = Path.cwd()

print(f"📁 Base directory: {base_dir}")

📁 Base directory: c:\Users\Olale\Documents\Codebase\Science\cafa-6-protein-function-prediction


## 1. Configuration

In [5]:
# ========================================
# MODEL CONFIGURATION
# ========================================
# Options: 
#   "facebook/esm2_t6_8M_UR50D"    - 8M params, fast (~1h training)
#   "facebook/esm2_t30_150M_UR50D" - 150M params, ~4h training
#   "facebook/esm2_t33_650M_UR50D" - 650M params, ~60h training
MODEL_NAME = "facebook/esm2_t30_150M_UR50D"  # 🔥 UPGRADED TO 150M

# ========================================
# TRAINING CONFIGURATION (adjusted for 150M)
# ========================================
# Memory-optimised settings for 150M on 8GB VRAM
BATCH_SIZE = 2                    # Reduced from 8 (150M needs more memory)
GRADIENT_ACCUMULATION = 16        # Increased to maintain effective batch size 32
LEARNING_RATE = 1e-5              # Slightly lower for larger model
NUM_EPOCHS = 10
MAX_LENGTH = 448                  # Reduced from 512 to save memory
USE_FP16 = True                   # Mixed precision for memory efficiency

# ========================================
# VOCABULARY CONFIGURATION
# ========================================
VOCAB_SIZE = 5000
MIN_COUNT = 10

# ========================================
# REGULARISATION
# ========================================
PATIENCE = 3
DROPOUT = 0.3

# Paths
SAVE_DIR = base_dir / "models" / "esm_finetuned_150M" if ENVIRONMENT == 'local' else Path("/kaggle/working/models/esm_finetuned_150M")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

print(f"🔥 Model: {MODEL_NAME}")
print(f"   Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"   Max sequence length: {MAX_LENGTH}")
print(f"   Mixed precision (fp16): {USE_FP16}")
print(f"   Vocabulary: Top {VOCAB_SIZE} GO terms")
print(f"   Save directory: {SAVE_DIR}")

🔥 Model: facebook/esm2_t30_150M_UR50D
   Effective batch size: 32
   Max sequence length: 448
   Mixed precision (fp16): True
   Vocabulary: Top 5000 GO terms
   Save directory: c:\Users\Olale\Documents\Codebase\Science\cafa-6-protein-function-prediction\models\esm_finetuned_150M


## 2. Load Data and Build Vocabulary

In [6]:
# Load sequences
print("Loading sequences...")
sequences = {}
for record in SeqIO.parse(base_dir / "Train" / "train_sequences.fasta", "fasta"):
    seq_id = record.id
    # Parse pipe-delimited format: sp|P53919|NAF1_YEAST -> P53919
    if '|' in seq_id:
        seq_id = seq_id.split('|')[1]
    sequences[seq_id] = str(record.seq)

print(f"Loaded {len(sequences)} sequences")
print(f"Sample IDs: {list(sequences.keys())[:3]}")

# Load annotations
print("Loading annotations...")
train_terms = pd.read_csv(base_dir / "Train" / "train_terms.tsv", sep='\t')
print(f"Total annotations: {len(train_terms)}")

Loading sequences...
Loaded 82404 sequences
Sample IDs: ['A0A0C5B5G6', 'A0JNW5', 'A0JP26']
Loading annotations...
Total annotations: 537027
Loaded 82404 sequences
Sample IDs: ['A0A0C5B5G6', 'A0JNW5', 'A0JP26']
Loading annotations...
Total annotations: 537027


In [7]:
# Build vocabulary: top N most frequent GO terms
print("Building vocabulary...")
term_counts = Counter(train_terms['term'])
filtered_terms = {term: count for term, count in term_counts.items() if count >= MIN_COUNT}
vocab_terms = [term for term, _ in sorted(filtered_terms.items(), key=lambda x: x[1], reverse=True)[:VOCAB_SIZE]]
vocab = {term: idx for idx, term in enumerate(vocab_terms)}

print(f"Vocabulary size: {len(vocab)}")
print(f"Coverage: {sum(filtered_terms[t] for t in vocab_terms) / len(train_terms):.2%} of annotations")

# Save vocabulary
with open(SAVE_DIR / "vocab.json", "w") as f:
    json.dump(vocab, f, indent=2)
print(f"✅ Vocabulary saved")

# Load IA weights for evaluation
print("\nLoading IA weights...")
ia_df = pd.read_csv(base_dir / "IA.tsv", sep='\t', header=None, names=['term', 'IA'])
ia_weights = dict(zip(ia_df['term'], ia_df['IA']))
print(f"✅ IA weights loaded: {len(ia_weights)} terms")

# Verify ID alignment
print("\n🔍 Checking ID alignment...")
terms_ids = set(train_terms['EntryID'].unique())
seq_ids = set(sequences.keys())
matching_ids = terms_ids & seq_ids
print(f"Annotation IDs: {len(terms_ids):,}")
print(f"Sequence IDs: {len(seq_ids):,}")
print(f"Matching IDs: {len(matching_ids):,}")
if len(matching_ids) == 0:
    print("⚠️ WARNING: No matching IDs! Check format alignment.")
else:
    print(f"✅ ID alignment verified ({100*len(matching_ids)/len(terms_ids):.1f}% coverage)")

Building vocabulary...
Vocabulary size: 5000
Coverage: 81.96% of annotations
✅ Vocabulary saved

Loading IA weights...
✅ IA weights loaded: 40122 terms

🔍 Checking ID alignment...
Annotation IDs: 82,404
Sequence IDs: 82,404
Matching IDs: 82,404
✅ ID alignment verified (100.0% coverage)
Vocabulary size: 5000
Coverage: 81.96% of annotations
✅ Vocabulary saved

Loading IA weights...
✅ IA weights loaded: 40122 terms

🔍 Checking ID alignment...
Annotation IDs: 82,404
Sequence IDs: 82,404
Matching IDs: 82,404
✅ ID alignment verified (100.0% coverage)


## 3. Create Dataset Class

In [8]:
class ProteinDataset(Dataset):
    def __init__(self, protein_ids, sequences_dict, annotations_df, vocab_dict, tokenizer, max_length=MAX_LENGTH):
        self.protein_ids = protein_ids
        self.sequences = sequences_dict
        self.vocab = vocab_dict
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        # Build protein -> terms mapping
        self.protein_terms = {}
        for protein_id in protein_ids:
            terms = annotations_df[annotations_df['EntryID'] == protein_id]['term'].tolist()
            # Filter to vocabulary
            terms_in_vocab = [t for t in terms if t in vocab_dict]
            self.protein_terms[protein_id] = terms_in_vocab
    
    def __len__(self):
        return len(self.protein_ids)
    
    def __getitem__(self, idx):
        protein_id = self.protein_ids[idx]
        sequence = self.sequences[protein_id]
        
        # Tokenize sequence
        tokens = self.tokenizer(
            sequence,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Create multi-hot label vector
        labels = torch.zeros(len(self.vocab), dtype=torch.float32)
        for term in self.protein_terms[protein_id]:
            labels[self.vocab[term]] = 1.0
        
        return {
            'input_ids': tokens['input_ids'].squeeze(0),
            'attention_mask': tokens['attention_mask'].squeeze(0),
            'labels': labels
        }

print(f"✅ Dataset class defined (max_length={MAX_LENGTH})")

✅ Dataset class defined (max_length=448)


## 4. Create Model Class

In [9]:
class ESMForGOPrediction(nn.Module):
    def __init__(self, model_name, num_labels, dropout=0.1):
        super().__init__()
        self.esm = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.esm.config.hidden_size, num_labels)
    
    def forward(self, input_ids, attention_mask):
        # Get ESM-2 embeddings
        outputs = self.esm(input_ids=input_ids, attention_mask=attention_mask)
        
        # Mean pooling
        token_embeddings = outputs.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        pooled = sum_embeddings / sum_mask
        
        # Classification
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)
        return logits

print("✅ Model class defined")

✅ Model class defined


## 5. Create Train/Val Split

In [10]:
# Split proteins
all_proteins = [p for p in train_terms['EntryID'].unique() if p in sequences]
train_proteins, val_proteins = train_test_split(
    all_proteins, test_size=0.2, random_state=42
)

print(f"Train proteins: {len(train_proteins)}")
print(f"Val proteins: {len(val_proteins)}")

Train proteins: 65923
Val proteins: 16481


In [11]:
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Create datasets
print("Creating datasets...")
train_dataset = ProteinDataset(train_proteins, sequences, train_terms, vocab, tokenizer)
val_dataset = ProteinDataset(val_proteins, sequences, train_terms, vocab, tokenizer)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"✅ Train batches: {len(train_loader)}")
print(f"✅ Val batches: {len(val_loader)}")

Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

c:\venv\cafa6\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Olale\.cache\huggingface\hub\models--facebook--esm2_t30_150M_UR50D. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Creating datasets...
✅ Train batches: 32962
✅ Val batches: 8241


## 6. Initialize Model and Optimizer

In [12]:
class AsymmetricLoss(nn.Module):
    """
    Asymmetric Loss for imbalanced multi-label classification.
    
    Down-weights easy negatives to focus model on hard positives.
    Used in archive's successful F1=0.23 run.
    
    Args:
        gamma_neg: Focusing parameter for negatives (higher = more down-weighting of easy negatives)
        gamma_pos: Focusing parameter for positives (usually 0)
        clip: Clipping threshold for negative probabilities
    """
    def __init__(self, gamma_neg=2.0, gamma_pos=0.0, clip=0.05):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
    
    def forward(self, logits, labels):
        # Get probabilities
        probs = torch.sigmoid(logits)
        
        # Asymmetric clipping for negatives
        probs_pos = probs
        probs_neg = (1 - probs).clamp(max=1 - self.clip)
        
        # Asymmetric focusing
        # Positive loss: standard cross-entropy with optional focusing
        pos_loss = -labels * torch.log(probs_pos.clamp(min=1e-8)) * ((1 - probs_pos) ** self.gamma_pos)
        
        # Negative loss: down-weight easy negatives (high confidence correct rejections)
        neg_loss = -(1 - labels) * torch.log(probs_neg.clamp(min=1e-8)) * (probs ** self.gamma_neg)
        
        loss = pos_loss + neg_loss
        return loss.mean()

print("✅ AsymmetricLoss class defined")

✅ AsymmetricLoss class defined


In [13]:
# Initialize model
print(f"Initializing model: {MODEL_NAME}...")
model = ESMForGOPrediction(MODEL_NAME, len(vocab), dropout=DROPOUT).to(device)

# Loss function - AsymmetricLoss for imbalanced multi-label classification
criterion = AsymmetricLoss(gamma_neg=2.0, gamma_pos=0.0, clip=0.05)
print("Using AsymmetricLoss (gamma_neg=2.0, gamma_pos=0.0)")

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler
total_steps = len(train_loader) * NUM_EPOCHS // GRADIENT_ACCUMULATION
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

# Mixed precision scaler (for fp16)
if USE_FP16:
    scaler = torch.cuda.amp.GradScaler()
    print("✅ Mixed precision (fp16) enabled")
else:
    scaler = None

param_count = sum(p.numel() for p in model.parameters())
trainable_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✅ Model initialized")
print(f"   Total parameters: {param_count:,}")
print(f"   Trainable parameters: {trainable_count:,}")

Initializing model: facebook/esm2_t30_150M_UR50D...


config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using AsymmetricLoss (gamma_neg=2.0, gamma_pos=0.0)
✅ Mixed precision (fp16) enabled
✅ Model initialized
   Total parameters: 151,343,841
   Trainable parameters: 151,343,841


C:\Users\Olale\AppData\Local\Temp\ipykernel_37224\1285750156.py:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [14]:
# 🔍 DIAGNOSTIC: Calculate actual positive rate
print("🔍 Analyzing class balance...")

total_labels = 0
positive_labels = 0

for protein_id in train_proteins[:1000]:  # Sample 1000 proteins
    terms = train_terms[train_terms['EntryID'] == protein_id]['term'].tolist()
    terms_in_vocab = [t for t in terms if t in vocab]
    total_labels += len(vocab)
    positive_labels += len(terms_in_vocab)

positive_rate = positive_labels / total_labels
print(f"\nPositive rate: {positive_rate:.4f} ({100*positive_rate:.2f}%)")
print(f"Negative rate: {1-positive_rate:.4f} ({100*(1-positive_rate):.2f}%)")
print(f"Imbalance ratio: 1 : {(1-positive_rate)/positive_rate:.1f}")
print(f"\n💡 Recommended pos_weight: {(1-positive_rate)/positive_rate:.1f}")
print(f"   (Currently using: 50.0)")


🔍 Analyzing class balance...

Positive rate: 0.0011 (0.11%)
Negative rate: 0.9989 (99.89%)
Imbalance ratio: 1 : 897.6

💡 Recommended pos_weight: 897.6
   (Currently using: 50.0)


## 7. Training Loop

In [15]:
def evaluate_model(model, dataloader, criterion, device, vocab_list, train_terms_df, ia_weights_dict, threshold=0.5):
    """
    Evaluate model with per-aspect CAFA metric (MF, BP, CC split).
    
    Args:
        model: trained model
        dataloader: validation dataloader
        criterion: loss function
        device: cuda/cpu
        vocab_list: list of GO terms in vocabulary order
        train_terms_df: DataFrame with 'term' and 'aspect' columns
        ia_weights_dict: dict mapping GO terms to IA weights
        threshold: prediction threshold
    
    Returns:
        tuple: (avg_loss, overall_f1, aspect_breakdown)
    """
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            
            total_loss += loss.item()
            
            # Get probabilities
            probs = torch.sigmoid(logits).cpu().numpy()
            all_preds.append(probs)
            all_labels.append(labels.cpu().numpy())
    
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    # Map vocab terms to aspects
    term_to_aspect = dict(zip(train_terms_df['term'], train_terms_df['aspect']))
    aspect_map = {'F': 'MF', 'P': 'BP', 'C': 'CC'}
    
    # Get aspect indices for each vocab term
    vocab_aspects = {}
    for idx, term in enumerate(vocab_list):
        aspect_letter = term_to_aspect.get(term, None)
        if aspect_letter:
            aspect_name = aspect_map.get(aspect_letter)
            if aspect_name:
                if aspect_name not in vocab_aspects:
                    vocab_aspects[aspect_name] = []
                vocab_aspects[aspect_name].append(idx)
    
    # Compute per-aspect F1
    aspect_results = {}
    
    for aspect_name, aspect_indices in vocab_aspects.items():
        if not aspect_indices:
            continue
        
        # Extract predictions and labels for this aspect
        aspect_preds = all_preds[:, aspect_indices]
        aspect_labels = all_labels[:, aspect_indices]
        aspect_terms = [vocab_list[i] for i in aspect_indices]
        
        # Threshold predictions
        aspect_preds_binary = (aspect_preds >= threshold).astype(int)
        
        # Get IA weights for this aspect's terms
        aspect_weights = np.array([ia_weights_dict.get(term, 1.0) for term in aspect_terms])
        
        # Compute per-sample F1
        f1_scores = []
        for i in range(len(aspect_labels)):
            true_pos = (aspect_labels[i] == 1) & (aspect_preds_binary[i] == 1)
            pred_pos = (aspect_preds_binary[i] == 1)
            actual_pos = (aspect_labels[i] == 1)
            
            tp = (true_pos * aspect_weights).sum()
            fp = ((pred_pos & ~true_pos) * aspect_weights).sum()
            fn = ((actual_pos & ~true_pos) * aspect_weights).sum()
            
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            
            f1_scores.append(f1)
        
        aspect_results[aspect_name] = np.mean(f1_scores)
    
    # Overall F1 = mean of aspect F1s
    overall_f1 = np.mean(list(aspect_results.values())) if aspect_results else 0.0
    
    return total_loss / len(dataloader), overall_f1, aspect_results

print("✅ Evaluation function defined (per-aspect CAFA metric)")

def evaluate_model_with_aspect_thresholds(model, dataloader, criterion, device, vocab_list, train_terms_df, ia_weights_dict,
                                          mf_threshold=0.30, bp_threshold=0.01, cc_threshold=0.30):
    """
    Evaluate model with per-aspect thresholds (different threshold for each aspect).
    
    Args:
        model: trained model
        dataloader: validation dataloader  
        criterion: loss function
        device: cuda/cpu
        vocab_list: list of GO terms in vocabulary order
        train_terms_df: DataFrame with 'term' and 'aspect' columns
        ia_weights_dict: dict mapping GO terms to IA weights
        mf_threshold: threshold for MF predictions
        bp_threshold: threshold for BP predictions (typically lower)
        cc_threshold: threshold for CC predictions
    
    Returns:
        tuple: (avg_loss, overall_f1, aspect_breakdown)
    """
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            
            total_loss += loss.item()
            
            # Get probabilities
            probs = torch.sigmoid(logits).cpu().numpy()
            all_preds.append(probs)
            all_labels.append(labels.cpu().numpy())
    
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    # Map vocab terms to aspects
    term_to_aspect = dict(zip(train_terms_df['term'], train_terms_df['aspect']))
    aspect_map = {'F': 'MF', 'P': 'BP', 'C': 'CC'}
    
    # Get aspect indices for each vocab term
    vocab_aspects = {'MF': [], 'BP': [], 'CC': []}
    for idx, term in enumerate(vocab_list):
        aspect_letter = term_to_aspect.get(term, None)
        if aspect_letter:
            aspect_name = aspect_map.get(aspect_letter)
            if aspect_name:
                vocab_aspects[aspect_name].append(idx)
    
    # Per-aspect thresholds
    thresholds = {'MF': mf_threshold, 'BP': bp_threshold, 'CC': cc_threshold}
    
    # Compute per-aspect F1
    aspect_results = {}
    
    for aspect_name, aspect_indices in vocab_aspects.items():
        if not aspect_indices:
            continue
        
        threshold = thresholds[aspect_name]
        
        # Extract predictions and labels for this aspect
        aspect_preds = all_preds[:, aspect_indices]
        aspect_labels = all_labels[:, aspect_indices]
        aspect_terms = [vocab_list[i] for i in aspect_indices]
        
        # Threshold predictions with aspect-specific threshold
        aspect_preds_binary = (aspect_preds >= threshold).astype(int)
        
        # Get IA weights for this aspect's terms
        aspect_weights = np.array([ia_weights_dict.get(term, 1.0) for term in aspect_terms])
        
        # Compute per-sample F1
        f1_scores = []
        for i in range(len(aspect_labels)):
            true_pos = (aspect_labels[i] == 1) & (aspect_preds_binary[i] == 1)
            pred_pos = (aspect_preds_binary[i] == 1)
            actual_pos = (aspect_labels[i] == 1)
            
            tp = (true_pos * aspect_weights).sum()
            fp = ((pred_pos & ~true_pos) * aspect_weights).sum()
            fn = ((actual_pos & ~true_pos) * aspect_weights).sum()
            
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            
            f1_scores.append(f1)
        
        aspect_results[aspect_name] = np.mean(f1_scores)
    
    # Overall F1 = mean of aspect F1s
    overall_f1 = np.mean(list(aspect_results.values())) if aspect_results else 0.0
    
    return total_loss / len(dataloader), overall_f1, aspect_results

print("✅ Per-aspect threshold evaluation function defined")


✅ Evaluation function defined (per-aspect CAFA metric)
✅ Per-aspect threshold evaluation function defined


In [ ]:
print("Starting training...\n")
print(f"🔥 Model: {MODEL_NAME}")
print(f"   Batch size: {BATCH_SIZE} x {GRADIENT_ACCUMULATION} = {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Mixed precision: {USE_FP16}")

best_f1 = 0
patience_counter = 0
history = []

for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"{'='*60}")
    
    # Training
    model.train()
    train_loss = 0
    optimizer.zero_grad()
    
    progress = tqdm(train_loader, desc="Training")
    for batch_idx, batch in enumerate(progress):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass with mixed precision
        if USE_FP16 and scaler is not None:
            with torch.cuda.amp.autocast():
                logits = model(input_ids, attention_mask)
                loss = criterion(logits, labels)
                loss = loss / GRADIENT_ACCUMULATION
            
            # Backward pass with gradient scaling
            scaler.scale(loss).backward()
            
            # Update weights every GRADIENT_ACCUMULATION steps
            if (batch_idx + 1) % GRADIENT_ACCUMULATION == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()
        else:
            # Standard training (no fp16)
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            loss = loss / GRADIENT_ACCUMULATION
            loss.backward()
            
            if (batch_idx + 1) % GRADIENT_ACCUMULATION == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
        
        train_loss += loss.item() * GRADIENT_ACCUMULATION
        progress.set_postfix({'loss': train_loss / (batch_idx + 1)})
    
    avg_train_loss = train_loss / len(train_loader)
    
    # Validation with per-aspect thresholds
    val_loss, val_f1, aspect_breakdown = evaluate_model_with_aspect_thresholds(
        model, val_loader, criterion, device, 
        vocab_terms, train_terms, ia_weights,
        mf_threshold=0.40, bp_threshold=0.20, cc_threshold=0.40
    )
    
    print(f"\nTrain Loss: {avg_train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Val F1 (Overall): {val_f1:.4f}")
    print(f"  MF: {aspect_breakdown.get('MF', 0):.4f} (threshold=0.40)")
    print(f"  BP: {aspect_breakdown.get('BP', 0):.4f} (threshold=0.20)")
    print(f"  CC: {aspect_breakdown.get('CC', 0):.4f} (threshold=0.40)")
    
    history.append({
        'epoch': epoch + 1,
        'train_loss': avg_train_loss,
        'val_loss': val_loss,
        'val_f1': val_f1,
        'val_f1_mf': aspect_breakdown.get('MF', 0),
        'val_f1_bp': aspect_breakdown.get('BP', 0),
        'val_f1_cc': aspect_breakdown.get('CC', 0)
    })
    
    # Save best model
    if val_f1 > best_f1:
        best_f1 = val_f1
        patience_counter = 0
        
        best_model_dir = SAVE_DIR / "best_model"
        best_model_dir.mkdir(exist_ok=True)
        torch.save(model.state_dict(), best_model_dir / "pytorch_model.bin")
        
        # Save config
        config = {
            'model_name': MODEL_NAME,
            'num_labels': len(vocab),
            'dropout': DROPOUT,
            'max_length': MAX_LENGTH,
            'best_f1': float(best_f1),
            'optimal_thresholds': {'MF': 0.40, 'BP': 0.20, 'CC': 0.40},
            'aspect_breakdown': {k: float(v) for k, v in aspect_breakdown.items()}
        }
        with open(best_model_dir / "config.json", "w") as f:
            json.dump(config, f, indent=2)
        
        print(f"🏆 New best model saved! F1: {best_f1:.4f}")
    else:
        patience_counter += 1
        print(f"No improvement. Patience: {patience_counter}/{PATIENCE}")
    
    # Early stopping
    if patience_counter >= PATIENCE:
        print(f"\n⏹️ Early stopping triggered after {epoch+1} epochs")
        break

print(f"\n{'='*60}")
print(f"✅ Training complete!")
print(f"   Best F1: {best_f1:.4f}")
print(f"   Model saved to: {SAVE_DIR / 'best_model'}")
print(f"{'='*60}")

Starting training...

🔥 Model: facebook/esm2_t30_150M_UR50D
   Batch size: 2 x 16 = 32
   Learning rate: 1e-05
   Mixed precision: True

Epoch 1/10


Training:   0%|          | 0/32962 [00:00<?, ?it/s]

C:\Users\Olale\AppData\Local\Temp\ipykernel_37224\1010690236.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


In [ ]:
# 🔍 DIAGNOSTIC: Check predictions
print("🔍 Checking predictions on validation set...")

model.eval()
with torch.no_grad():
    # Get one batch
    batch = next(iter(val_loader))
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    
    logits = model(input_ids, attention_mask)
    probs = torch.sigmoid(logits)
    
    print(f"\nBatch shape: {probs.shape}")
    print(f"Prediction stats:")
    print(f"  Min: {probs.min().item():.6f}")
    print(f"  Max: {probs.max().item():.6f}")
    print(f"  Mean: {probs.mean().item():.6f}")
    print(f"  Median: {probs.median().item():.6f}")
    
    print(f"\nLabel stats:")
    print(f"  Positive labels per sample: {labels.sum(dim=1).mean().item():.1f}")
    print(f"  Total positive labels: {labels.sum().item():.0f}")
    
    # Check predictions at different thresholds
    for thresh in [0.1, 0.3, 0.5, 0.7]:
        preds_count = (probs >= thresh).sum().item()
        print(f"  Predictions >= {thresh}: {preds_count}")
    
    # Check if any predictions match labels
    preds_05 = (probs >= 0.5).float()
    matches = (preds_05 == labels).sum().item()
    total = labels.numel()
    print(f"\nExact matches at 0.5 threshold: {matches}/{total} ({100*matches/total:.1f}%)")

## 8. Save Training History

In [ ]:
# Save history
history_df = pd.DataFrame(history)
history_df.to_csv(SAVE_DIR / "training_history.csv", index=False)

print("\n📊 Training History:")
print(history_df.to_string(index=False))

print(f"\n✅ Model and history saved to {SAVE_DIR}")

## Summary

**ESM-2 150M Fine-Tuned Classifier:**
- Model: `facebook/esm2_t30_150M_UR50D` (150M parameters)
- Trained on 5000 most common GO terms
- Multi-label classification with AsymmetricLoss
- Mixed precision (fp16) for memory efficiency
- Per-aspect thresholds: MF=0.40, BP=0.20, CC=0.40
- Expected F1: ~0.34-0.37

**Memory Optimisation:**
- Batch size 2 (with gradient accumulation 16)
- Max sequence length 448 (reduced from 512)
- FP16 mixed precision training

**Next Steps:**
1. Scale to ESM-2 650M if 150M successful
2. Generate test predictions for submission
3. Build ensemble with KNN